In [1]:
import re
import pandas as pd
import numpy as np
import src.functions as f
from pymongo import MongoClient
client = MongoClient("localhost:27017")

In [2]:
db = client.get_database("ironhack")

In [3]:
companies = db.get_collection("companies")

All the companies dedicated to design

All the tech companies that have raised at least 1 Million dollars

In [4]:
query_1 = {'$and' : [{"tag_list" : {'$regex':'design'}}, {"tag_list" : {'$regex':'(?s)^((?!fashion).)*'}}]}
query_2 = {'$and' : [{"total_money_raised" : {'$regex':'M'}},{"tag_list" : {'$regex':'tech'}}]}
proj = {'name': 1, '_id': 0, 'total_money_raised' : 1, 'offices.city' : 1, 'offices.latitude': 1, 'offices.longitude': 1}
companies_filtered = list(companies.find({'$or' : [query_2, query_1]}, proj))

In [5]:
companies_filtered[0]

{'name': 'Digg',
 'total_money_raised': '$45M',
 'offices': [{'city': 'San Francisco',
   'latitude': 37.764726,
   'longitude': -122.394523}]}

In [6]:
name = []
city = []
latitude = []
longitude = []
for i in companies_filtered:
    name.append(i['name'])
    try: 
        if i['offices'][0]['city'] == '':
            city.append(np.nan)
        else:
            city.append(i['offices'][0]['city'])
        latitude.append(i['offices'][0]['latitude'])
        longitude.append(i['offices'][0]['longitude'])
    except:
        city.append(np.nan)
        latitude.append(np.nan)
        longitude.append(np.nan)
        


In [16]:
dict_ = {'company' : name, 'city' : city, 'latitude' : latitude, 'longitude': longitude}

In [8]:
companies_df = pd.DataFrame.from_dict(dict_, orient='columns')

In [9]:
dict_of_cities = f.vc_to_dict(companies_df.city)

In [10]:
cities = f.subdata(11,dict_of_cities)

In [11]:
cities

['San Francisco', 'New York', 'London']

In [12]:
df = f.create(companies_df,'city',cities)

In [13]:
df

,company,city,latitude,longitude
0,Digg,San Francisco,37.764726,-122.394523
1,Powerset,San Francisco,37.778613,-122.395289
2,CastTV,San Francisco,37.780716,-122.393913
4,Grockit,San Francisco,37.775196,-122.419204
8,Wikia,San Francisco,NaN,NaN
...,...,...,...,...
737,Synesis IT,New York,40.748873,-73.987106
740,Ramp Up Tech,New York,40.746514,-73.993035
779,OpTier,New York,40.754937,-73.985647
787,Mobile Interactive Group,London,NaN,NaN


In [15]:
df.to_csv("data/df.csv")